In [1]:
import pandas as pd 

stock_data=pd.read_html("https://www.finanznachrichten.de/nachrichten/alle-empfehlungen.html")



In [2]:

len(stock_data)

4

In [3]:
stock_data_first=pd.DataFrame(stock_data[1])
print(stock_data_first.head())
stock_data_first.columns

  Zeit                               Aktuelle Nachrichten  \
0   Fr        GOLDMAN SACHS belässt SOFTWARE AG auf 'Buy'   
1   Fr         JEFFERIES belässt CONTINENTAL AG auf 'Buy'   
2   Fr                GOLDMAN SACHS belässt ENI auf 'Buy'   
3   Fr  GOLDMAN SACHS belässt MTU Aero Engines AG auf ...   
4   Fr                 RBC belässt BARCLAYS auf 'Neutral'   

  Unternehmen / Aktien        PotentialKurse aktuell  Leser  Unnamed: 5  
0          SOFTWARE AG    +22 % (45 Euro)36,920 Euro  2.360         NaN  
1       CONTINENTAL AG   +29 % (155 Euro)120,35 Euro  2.392         NaN  
2              ENI SPA  +25 % (11,50 Euro)9,200 Euro  2.253         NaN  
3  MTU AERO ENGINES AG    -6 % (180 Euro)191,55 Euro  1.883         NaN  
4         BARCLAYS PLC     -2 % (1,50 GBP)1,775 Euro  1.769         NaN  


Index(['Zeit', 'Aktuelle Nachrichten', 'Unternehmen / Aktien',
       'PotentialKurse aktuell', 'Leser', 'Unnamed: 5'],
      dtype='object')

In [4]:
import re

stock_data_first['Aktuelle Nachrichten']=stock_data_first['Aktuelle Nachrichten'].str.findall('(?:stuft|belässt)\s([A-Za-z]+)').transform(''.join)
stock_data_first.head()

,Zeit,Aktuelle Nachrichten,Unternehmen / Aktien,PotentialKurse aktuell,Leser,Unnamed: 5
0,Fr,SOFTWARE,SOFTWARE AG,"+22 % (45 Euro)36,920 Euro",2.360,NaN
1,Fr,CONTINENTAL,CONTINENTAL AG,"+29 % (155 Euro)120,35 Euro",2.392,NaN
2,Fr,ENI,ENI SPA,"+25 % (11,50 Euro)9,200 Euro",2.253,NaN
3,Fr,MTU,MTU AERO ENGINES AG,"-6 % (180 Euro)191,55 Euro",1.883,NaN
4,Fr,BARCLAYS,BARCLAYS PLC,"-2 % (1,50 GBP)1,775 Euro",1.769,NaN


In [5]:
stock_data_first['Potential in %']=stock_data_first['PotentialKurse aktuell'].str.findall('([+-]\d*)\s%').transform(''.join).apply(pd.to_numeric)


In [6]:
stock_data_first.head()

,Zeit,Aktuelle Nachrichten,Unternehmen / Aktien,PotentialKurse aktuell,Leser,Unnamed: 5,Potential in %
0,Fr,SOFTWARE,SOFTWARE AG,"+22 % (45 Euro)36,920 Euro",2.360,NaN,22.0
1,Fr,CONTINENTAL,CONTINENTAL AG,"+29 % (155 Euro)120,35 Euro",2.392,NaN,29.0
2,Fr,ENI,ENI SPA,"+25 % (11,50 Euro)9,200 Euro",2.253,NaN,25.0
3,Fr,MTU,MTU AERO ENGINES AG,"-6 % (180 Euro)191,55 Euro",1.883,NaN,-6.0
4,Fr,BARCLAYS,BARCLAYS PLC,"-2 % (1,50 GBP)1,775 Euro",1.769,NaN,-2.0


In [7]:
stock_data_first=stock_data_first.sort_values(by=['Potential in %'],ascending=False)
stock_data_first.head()

,Zeit,Aktuelle Nachrichten,Unternehmen / Aktien,PotentialKurse aktuell,Leser,Unnamed: 5,Potential in %
9,Fr,ENI,ENI SPA,"+36 % (12,50 Euro)9,200 Euro",2.545,NaN,36.0
1,Fr,CONTINENTAL,CONTINENTAL AG,"+29 % (155 Euro)120,35 Euro",2.392,NaN,29.0
2,Fr,ENI,ENI SPA,"+25 % (11,50 Euro)9,200 Euro",2.253,NaN,25.0
17,Fr,RWE,RWE AG,"+23 % (40 Euro)32,470 Euro",2.181,NaN,23.0
0,Fr,SOFTWARE,SOFTWARE AG,"+22 % (45 Euro)36,920 Euro",2.360,NaN,22.0


In [8]:
from bs4 import BeautifulSoup
import requests

page=requests.get("https://www.finanznachrichten.de/nachrichten/alle-empfehlungen.html")
soup=BeautifulSoup(page.content,"html.parser")

In [9]:
best_five=stock_data_first.iloc[:5]

In [10]:
best_five

,Zeit,Aktuelle Nachrichten,Unternehmen / Aktien,PotentialKurse aktuell,Leser,Unnamed: 5,Potential in %
9,Fr,ENI,ENI SPA,"+36 % (12,50 Euro)9,200 Euro",2.545,NaN,36.0
1,Fr,CONTINENTAL,CONTINENTAL AG,"+29 % (155 Euro)120,35 Euro",2.392,NaN,29.0
2,Fr,ENI,ENI SPA,"+25 % (11,50 Euro)9,200 Euro",2.253,NaN,25.0
17,Fr,RWE,RWE AG,"+23 % (40 Euro)32,470 Euro",2.181,NaN,23.0
0,Fr,SOFTWARE,SOFTWARE AG,"+22 % (45 Euro)36,920 Euro",2.360,NaN,22.0


In [11]:
tables_list=[x for x in soup.find_all("table",attrs={"class":"rel-ct"})]

In [12]:
links=tables_list[0].tbody.find_all('a')
filtered_links=[x.get('href') for i,x in enumerate(links) if i%2==0]


In [32]:
best_five['links']=best_five['Aktuelle Nachrichten'].apply(lambda x: [y for y in filtered_links if x.lower() in y][0])
best_five

,Zeit,Aktuelle Nachrichten,Unternehmen / Aktien,PotentialKurse aktuell,Leser,Unnamed: 5,Potential in %,links
9,Fr,ENI,ENI SPA,"+36 % (12,50 Euro)9,200 Euro",2.545,NaN,36.0,https://www.finanznachrichten.de/nachrichten-2...
1,Fr,CONTINENTAL,CONTINENTAL AG,"+29 % (155 Euro)120,35 Euro",2.392,NaN,29.0,https://www.finanznachrichten.de/nachrichten-2...
2,Fr,ENI,ENI SPA,"+25 % (11,50 Euro)9,200 Euro",2.253,NaN,25.0,https://www.finanznachrichten.de/nachrichten-2...
17,Fr,RWE,RWE AG,"+23 % (40 Euro)32,470 Euro",2.181,NaN,23.0,https://www.finanznachrichten.de/nachrichten-2...
0,Fr,SOFTWARE,SOFTWARE AG,"+22 % (45 Euro)36,920 Euro",2.360,NaN,22.0,https://www.finanznachrichten.de/nachrichten-2...


In [42]:
isins=[]
for index,li in best_five.iterrows():
    page_2=requests.get(li['links'])
    soup_2=BeautifulSoup(page_2.content,"html.parser")
    span=soup_2.find('span',attrs={'class':"wkn-isin"})
    isins.append((span.find_all(id="produkt-isin")[0].get('data-isin'),li['links'],li['Potential in %'],li['Unternehmen / Aktien']))

In [43]:
isins

[('IT0003132476',
  'https://www.finanznachrichten.de/nachrichten-2021-02/52072181-goldman-sachs-belaesst-eni-auf-buy-322.htm',
  36.0,
  'ENI SPA'),
 ('DE0005439004',
  'https://www.finanznachrichten.de/nachrichten-2021-02/52072406-jefferies-belaesst-continental-ag-auf-buy-322.htm',
  29.0,
  'CONTINENTAL AG'),
 ('IT0003132476',
  'https://www.finanznachrichten.de/nachrichten-2021-02/52072181-goldman-sachs-belaesst-eni-auf-buy-322.htm',
  25.0,
  'ENI SPA'),
 ('DE0007037129',
  'https://www.finanznachrichten.de/nachrichten-2021-02/52068796-kepler-cheuvreux-belaesst-rwe-ag-auf-hold-322.htm',
  23.0,
  'RWE AG'),
 ('DE000A2GS401',
  'https://www.finanznachrichten.de/nachrichten-2021-02/52072474-goldman-sachs-belaesst-software-ag-auf-buy-322.htm',
  22.0,
  'SOFTWARE AG')]

In [46]:
stock_data_onvista=pd.DataFrame([])
for isin in isins:
    #page_3=requests.get("https://www.onvista.de/aktien/fundamental/-Aktie-{}".format(isin[0]))
    data=pd.read_html("https://www.onvista.de/aktien/fundamental/-Aktie-{}".format(isin[0]))
    filtered_data=data[1].rename(columns={data[1].columns[0]:"Stats"})
    filtered_data['Unternehmen']=isin[3]
    filtered_data['Potential_Analyse']=isin[2]
    filtered_data=filtered_data.set_index(filtered_data['Unternehmen']).iloc[2]
    stock_data_onvista=stock_data_onvista.append(filtered_data) 


In [53]:
stock_data_onvista.set_index([pd.Index([1, 2, 3, 4, 5])])

,2017,2018,2019,2020e,2021e,2022e,2023e,Potential_Analyse,Stats,Unternehmen,2020
1,-,"+22,34%","-96,52%",-,-,"+75,13%","+35,61%",36.0,Gewinnwachstum,ENI SPA,NaN
2,"+6,49%","-2,88%",-,"-48,55%",-,"+48,21%","+38,62%",29.0,Gewinnwachstum,CONTINENTAL AG,NaN
3,-,"+22,34%","-96,52%",-,-,"+75,13%","+35,61%",25.0,Gewinnwachstum,ENI SPA,NaN
4,"+66,52%","-82,52%","+2.459,25%","-88,47%","+21,18%","-7,26%","-19,26%",23.0,Gewinnwachstum,RWE AG,NaN
5,"+2,17%","+18,61%","-6,27%",NaN,"-9,71%","+29,65%","+27,42%",22.0,Gewinnwachstum,SOFTWARE AG,"-38,27%"


In [51]:
[x for x in range(5)]

[0, 1, 2, 3, 4]